In [39]:
!pip install transformers torch

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd

In [41]:
!wget https://indiaai.s3.ap-south-1.amazonaws.com/docs/test.csv
#!wget https://indiaai.s3.ap-south-1.amazonaws.com/docs/train.csv

--2024-11-05 02:42:37--  https://indiaai.s3.ap-south-1.amazonaws.com/docs/test.csv
Resolving indiaai.s3.ap-south-1.amazonaws.com (indiaai.s3.ap-south-1.amazonaws.com)... 52.219.158.170, 52.219.158.74, 52.219.156.66, ...
Connecting to indiaai.s3.ap-south-1.amazonaws.com (indiaai.s3.ap-south-1.amazonaws.com)|52.219.158.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14069068 (13M) [text/csv]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>]  13.42M  8.94MB/s    in 1.5s    

2024-11-05 02:42:40 (8.94 MB/s) - ‘test.csv.1’ saved [14069068/14069068]



In [51]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [52]:
df_train.describe()

,category,sub_category,crimeaditionalinfo
count,83437,83437,83437
unique,15,39,77858
top,Online Financial Fraud,UPI Related Frauds,fraud through upi fraud phone pe bank state ba...
freq,50794,22891,195


In [6]:
#df_train.loc[df_train["sub_category"].isnull(), "sub_category"] = df_train["category"]
#df_train= df_train.dropna(how='any')

In [7]:
#df_train.describe()

,category,sub_category,crimeaditionalinfo
count,93665,93665,93665
unique,15,39,85013
top,Online Financial Fraud,UPI Related Frauds,Respected Sir\r\n\r\nA very serious matter I w...
freq,57416,26843,2342


In [53]:
df_test.describe()

,category,sub_category,crimeaditionalinfo
count,31229,28993,31222
unique,15,37,28785
top,Online Financial Fraud,UPI Related Frauds,Respected Sir\r\n\r\nA very serious matter I w...
freq,18896,8890,753


In [54]:
df_test.loc[df_test["sub_category"].isnull(), "sub_category"] = df_test["category"]
df_test= df_test.dropna(how='any')

In [55]:
df_test.describe()

,category,sub_category,crimeaditionalinfo
count,31222,31222,31222
unique,15,41,28785
top,Online Financial Fraud,UPI Related Frauds,Respected Sir\r\n\r\nA very serious matter I w...
freq,18890,8886,753


In [47]:
sorted(df_train["sub_category"].unique()) == sorted(df_test["sub_category"].unique())

False

In [56]:
sub_categories = sorted(set(df_test["sub_category"].unique().tolist() + df_train["sub_category"].unique().tolist()))
d_cat = {}
for i, cat in enumerate(sub_categories):
  d_cat[cat] = i
d_cat

{'Against Interest of sovereignty or integrity of India': 0,
 'Business Email CompromiseEmail Takeover': 1,
 'Cheating by Impersonation': 2,
 'Child Pornography CPChild Sexual Abuse Material CSAM': 3,
 'Computer Generated CSAM/CSEM': 4,
 'Cryptocurrency Fraud': 5,
 'Cyber Blackmailing & Threatening': 6,
 'Cyber Bullying  Stalking  Sexting': 7,
 'Cyber Terrorism': 8,
 'Damage to computer computer systems etc': 9,
 'Data Breach/Theft': 10,
 'DebitCredit Card FraudSim Swap Fraud': 11,
 'DematDepository Fraud': 12,
 'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks': 13,
 'EMail Phishing': 14,
 'EWallet Related Fraud': 15,
 'Email Hacking': 16,
 'FakeImpersonating Profile': 17,
 'Fraud CallVishing': 18,
 'Hacking/Defacement': 19,
 'Impersonating Email': 20,
 'Internet Banking Related Fraud': 21,
 'Intimidating Email': 22,
 'Malware Attack': 23,
 'Online Gambling  Betting': 24,
 'Online Job Fraud': 25,
 'Online Matrimonial Fraud': 26,
 'Online Trafficking': 27,
 'Other':

In [57]:
df_train["subcat_label"] = df_train["sub_category"].apply(lambda x: d_cat.get(x, "unknown"))
df_test["subcat_label"] = df_test["sub_category"].apply(lambda x: d_cat.get(x, "unknown"))
train_text = df_train["crimeaditionalinfo"].tolist()
train_labels = df_train["subcat_label"].tolist()
test_text = df_test["crimeaditionalinfo"].tolist()
test_labels = df_test["subcat_label"].tolist()

In [50]:
# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=39)  # Set `num_labels` to the number of categories
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [36]:
from torch.utils.data import Dataset

# Custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize text
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        # Return input_ids and attention_mask as well as the label
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize dataset
train_set = TextClassificationDataset(train_text[:40000], train_labels[:40000], tokenizer) # change to feed full dataset
test_set = TextClassificationDataset(test_text[:8000], test_labels[:8000], tokenizer) # change to feed full dataset

In [37]:
len(train_set), len(test_set)

(40000, 8000)

In [38]:
# Training arguments
batch_size = 4
num_epochs = 2
train_dataset_size = len(train_set)
steps_per_epoch = train_dataset_size // batch_size

training_args = TrainingArguments(
    output_dir="./results",           # output directory
    overwrite_output_dir=True,        # output directory
    num_train_epochs=num_epochs,      # number of epochs
    per_device_train_batch_size=4,    # batch size for training
    per_device_eval_batch_size=4,     # batch size for evaluation
    eval_strategy="steps",            # evaluate every epoch
    logging_dir="./logs",
    logging_steps= 1000, #steps_per_epoch,
    save_strategy="epoch",            # save checkpoints at end of epoch
    save_total_limit=1,               # save only latest check point
    #max_steps=10,                     # Stop training after 10 steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [39]:
def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=-1).item()  # Get the index of the highest score
    return predicted_class

# Example prediction
print(f'crime description : {test_text[0]}')
print(f'actual category : {list(d_cat.items())[test_labels[0]]}')
new_text = test_text[2]
predicted_category = predict_category(new_text)
print(f"Predicted category: {list(d_cat.items())[predicted_category]}")

crime description : I had continue received random calls and abusive messages in my whatsapp Someone added my number in a unknown facebook group name with  Only Girls  and still getting calls from unknown numbers pls help me and sort out the issue  as soon as possible Thank you
actual category : ('Cyber Bullying  Stalking  Sexting', 5)
Predicted category: ('Other', 26)


In [32]:
test_text[2]

'He is acting like a police and demanding for money by adding sections in the text messages \r\nI request you to take an immediate action on him Frequently he is harassing with these messages For reference please find attached files'